#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 4, 14)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,19,S,SYNEX,"3,000",28.25,0,CP1S,SET100,2022-03-10


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
B,ASIAN,2,6L,SET,10000,15.000000,14.800000,0.000000,0.00%,1,2022-04-29,Buy,150000.000000
B,BAM,-1,3L,SET100,12000,19.800000,19.900000,-0.100000,-0.50%,1,2022-04-29,Buy,237600.000000
B,BBL,-4,1L,SET50,2400,133.000000,135.000000,-0.500000,-0.37%,1,2022-04-22,Buy,319200.000000
B,KKP,-4,3L,SET100,3000,68.000000,69.000000,-0.250000,-0.36%,1,2022-04-28,Buy,204000.000000
B,MAKRO,-4,RD05pct,SET,1500,38.000000,39.000000,-0.750000,-1.89%,1,2022-03-03,Buy,57000.000000


### End of Tables in the process

### Print to verify before upload file

In [7]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [8]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [9]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [10]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

Series([], Name: amount, dtype: object)

In [11]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [12]:
cash = 97_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(97000, 0, 97000)

In [13]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [14]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(97000, 0, 97000)

### After call ord-log (must call everytime that orders change)

In [15]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [16]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [17]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,BAM,-4,3L,SET100,12000,19.50,19.90,-0.10,-0.50%,1,2022-04-29,Buy,234000.0
2,B,BBL,-4,1L,SET50,2400,133.00,135.00,-0.50,-0.37%,1,2022-04-22,Buy,319200.0
3,B,KKP,-4,3L,SET100,3000,68.00,69.00,-0.25,-0.36%,1,2022-04-28,Buy,204000.0
4,B,MAKRO,-4,RD05pct,SET,1500,38.00,39.00,-0.75,-1.89%,1,2022-03-03,Buy,57000.0
5,B,SC,-4,HD,SET,30000,3.80,3.88,0.06,+1.57%,1,2022-04-29,Buy,114000.0
6,B,SENA,-4,RD05pct,SET,15000,4.48,4.56,-0.04,-0.87%,1,2022-05-10,Buy,67200.0


In [18]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [19]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
7,S,AIMIRT,4,DOS,SET,10000,13.1,12.7,-0.1,-0.78%,1,2022-03-14,Sell,131000.0
8,S,CPNCG,3,DOS,SET,20000,12.5,12.2,0.0,0.00%,1,2022-02-14,Sell,250000.0
12,S,JASIF,4,DOS,SET,10000,11.3,10.9,0.0,0.00%,1,2022-03-03,Sell,113000.0


In [20]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,BAM,-4,3L,SET100,12000,19.50,19.90,-0.10,-0.50%,1,2022-04-29,Buy,234000.0
2,B,BBL,-4,1L,SET50,2400,133.00,135.00,-0.50,-0.37%,1,2022-04-22,Buy,319200.0
3,B,KKP,-4,3L,SET100,3000,68.00,69.00,-0.25,-0.36%,1,2022-04-28,Buy,204000.0
4,B,MAKRO,-4,RD05pct,SET,1500,38.00,39.00,-0.75,-1.89%,1,2022-03-03,Buy,57000.0
5,B,SC,-4,HD,SET,30000,3.80,3.88,0.06,+1.57%,1,2022-04-29,Buy,114000.0
6,B,SENA,-4,RD05pct,SET,15000,4.48,4.56,-0.04,-0.87%,1,2022-05-10,Buy,67200.0
7,S,AIMIRT,4,DOS,SET,10000,13.10,12.70,-0.10,-0.78%,1,2022-03-14,Sell,131000.0
8,S,CPNCG,3,DOS,SET,20000,12.50,12.20,0.00,0.00%,1,2022-02-14,Sell,250000.0
12,S,JASIF,4,DOS,SET,10000,11.30,10.90,0.00,0.00%,1,2022-03-03,Sell,113000.0


In [21]:
df[mb | ms].shape[0]

9

In [22]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
2,BBL,Buy,"319,200.00",-4
1,BAM,Buy,"234,000.00",-4
3,KKP,Buy,"204,000.00",-4
0,ASIAN,Buy,"140,000.00",-8
5,SC,Buy,"114,000.00",-4


### Select source of orders between these two

In [23]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,BAM,12000,19.50,1,-4,19.90,-0.10,-0.50%,3L,SET100,2022-04-29
2,Buy,BBL,2400,133.00,1,-4,135.00,-0.50,-0.37%,1L,SET50,2022-04-22
3,Buy,KKP,3000,68.00,1,-4,69.00,-0.25,-0.36%,3L,SET100,2022-04-28
4,Buy,MAKRO,1500,38.00,1,-4,39.00,-0.75,-1.89%,RD05pct,SET,2022-03-03
5,Buy,SC,30000,3.80,1,-4,3.88,0.06,+1.57%,HD,SET,2022-04-29
6,Buy,SENA,15000,4.48,1,-4,4.56,-0.04,-0.87%,RD05pct,SET,2022-05-10
7,Sell,AIMIRT,10000,13.10,1,4,12.70,-0.10,-0.78%,DOS,SET,2022-03-14
8,Sell,CPNCG,20000,12.50,1,3,12.20,0.00,0.00%,DOS,SET,2022-02-14
12,Sell,JASIF,10000,11.30,1,4,10.90,0.00,0.00%,DOS,SET,2022-03-03


In [43]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,MAKRO,1500,39.00,2,-2,39.50,-0.25,-0.63%,RD05pct,SET,2022-03-03
2,Buy,SENA,15000,4.48,2,-5,4.58,-0.02,-0.43%,RD05pct,SET,2022-05-10
4,Sell,AIMIRT,10000,13.10,2,4,12.70,-0.10,-0.78%,DOS,SET,2022-03-14
5,Sell,CPNCG,20000,12.50,2,4,12.10,-0.10,-0.82%,DOS,SET,2022-02-14
6,Sell,DOHOME,2400,22.00,2,5,21.50,0.10,+0.47%,C21.3,SET100,2022-02-02
11,Sell,PTT,3000,39.00,2,5,37.75,0.00,0.00%,CP2S,SET50,2022-03-03


In [24]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process